In [4]:
import pandas as pd
import os
import re
from tqdm import tqdm

records = pd.read_excel(r"4.xlsx", header=[0], index_col = [0])

path = "Data_File/ismt_yices2"
folder_list = os.listdir(path)
for filefolder in folder_list:
    file_path = path + "/" + filefolder
    file_list = os.listdir(file_path)
    solved = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0}
    times = {'AProVE': 0, 'calypto': 0, 'Dartagnan': 0, 'LassoRanker': 0, 'leipzig': 0, 'mcm': 0, 'CInteger': 0, 'ITS': 0, 'SAT14': 0, 'total': 0}
    for filename in tqdm(file_list, desc = filefolder):
        print(filename)
        with open(os.path.join(file_path, filename)) as f:
            while f.readline():
                f.readline()
                flag = -1
                result = f.readline()
                while(result[:2] != 'is' or result[:2] != 'Fi'):
                    result = f.readline()
                if result[:4] == 'ismt':
                    info = f.readline()
                    if result[5] == 's':
                        result = 'sat'
                    elif result[5] == 'u':
                        result = 'unsat'
                else:
                    result = f.readline()
                    if result[:2] == '..':
                        info = result
                        result = 'timeout'
                    elif result[:2] == 'fi':
                        info = f.readline()
                        if result[13] == 's':
                            result = 'sat'
                        elif result[13] == 'u':
                            result = 'unsat'
                    elif result[:4] == 'unkn':
                        info = f.readline()
                        result = 'unknow'
                print(info)
                info = info.split('/')
                time = int(info[-1].split(' : ')[1][:-5])
                info[-1] = info[-1].split(' : ')[0]
                if result[:3] == 'sat':
                    solved['total'] += 1
                    times['total'] += time
                    if info[4] == '20170427-VeryMax':
                        if(len(info) == 5):
                            solved['SAT14'] += 1
                            times['SAT14'] += time
                        else:
                            solved[info[5]] += 1
                            times[info[5]] += time
                    elif info[4] == 'UltimateAutomizer' or info[4] == 'UltimateLassoRanker':
                        solved['LassoRanker'] += 1
                        times['LassoRanker'] += time
                    elif info[4] == '20210219-Dartagnan':
                        solved['Dartagnan'] += 1
                        times['Dartagnan'] += time
                    elif info[4] != '2019-ezsmt':
                        solved[info[4]] += 1
                        times[info[4]] += time
    records.loc[records['name'] == filefolder + '_solved', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total']] = [solved['AProVE'], solved['calypto'], solved['Dartagnan'], solved['LassoRanker'], solved['leipzig'], solved['mcm'], solved['CInteger'], solved['ITS'], solved['SAT14'], solved['total']]
    records.loc[records['name'] == filefolder + '_time', ['AProVE', 'calypto', 'Dartagnan', 'LassoRanker', 'leipzig', 'mcm', 'CInteger', 'ITS', 'SAT14', 'total']] = [times['AProVE'], times['calypto'], times['Dartagnan'], times['LassoRanker'], times['leipzig'], times['mcm'], times['CInteger'], times['ITS'], times['SAT14'], times['total']]
    # print(total)
print("Waiting...")
records.to_excel('Result/4_1.xlsx')
print("Done")    

20:   0%|          | 0/50 [00:00<?, ?it/s]

qfnia_2_20.log
../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394complete-fail.t2__p15157_safety_0.smt2 : 410 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394complete-fail.t2__p15160_safety_0.smt2 : 431 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394complete-fail.t2__p15161_safety_0.smt2 : 696 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394-fail.t2__p15221_safety_0.smt2 : 459 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394-fail.t2__p15222_safety_0.smt2 : 941 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394-succeed.t2__p15250_safety_0.smt2 : 479 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1394-succeed.t2__p15251_safety_0.smt2 : 1046 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1.t2__p15279_safety_0.smt2 : 132 ms.

../../QF_NIA_split/QF_NIA_2_20//20170427-VeryMax/ITS/From_T2__1.t2__p15280_safety_0.smt2 : 900 ms.

AttributeError: 'list' object has no attribute 'split'